In [1]:
categories = ['bitcoin',
              'smart-contract-platform', 
              'centralized-exchange-token-cex',
              'decentralized-finance-defi',
              'non-fungible-tokens-nft',
              'meme-token',
              'gaming',
              'Other',
              'sports'
             ]

In [2]:
from typing import Dict
def categs2IndexIds(categs: [str]) -> Dict[str, str]:
    result = {}
    for categ in categs:
        if categ != 'bitcoin' and categ != 'USDCUSDT':
            result[categ] = f"ONN-({categ})-B-30dW-30dR"
    return result

In [3]:
import pandas as pd
import httpx
host = 'http://localhost:9000'
async def getSymb(fromIsoInstant: str, toIsoInstant: str, symb: str) -> pd.DataFrame:
    def listToStr(l):
        result = l[0]
        for i in range(1, len(l)):
            result += f", {l[i]}"
        return result
    
    columns = ['symb', 'open_time', 'close', 'volume', 'close_time', 'quote_asset_vol']        
    sqlStmt = "SELECT %s FROM 'binance_daily' WHERE symb = '%s' AND open_time >= '%s' AND open_time < '%s'"%(listToStr(columns), symb, fromIsoInstant, toIsoInstant)

    query_params = {'query': sqlStmt, 'fmt': 'json'}
    async with httpx.AsyncClient() as client:
        r = await client.get(host + '/exec', params=query_params)
    jsonR = r.json()
    #print(jsonR)
    result = pd.DataFrame(columns=columns, data=jsonR['dataset'])
    result['strDate'] = result.apply(lambda row: row['open_time'][0:10], axis = 1)
    result.set_index('strDate', drop=True, inplace=True)
    result.rename(columns={'close':symb}, inplace=True)
    result = result[[symb]].sort_values('strDate')
    
    return result    

In [4]:
res = await getSymb('2021-01-01T00:00:00.000000Z', '2022-08-30T00:00:00.000000Z', 'USDCUSDT')
res

,USDCUSDT
strDate,
2021-01-01,0.9974
2021-01-02,0.9989
2021-01-03,0.9992
2021-01-04,0.9994
2021-01-05,0.9969
...,...
2022-08-25,1.0000
2022-08-26,1.0000
2022-08-27,1.0001


In [5]:
import pandas as pd
from typing import Dict
import httpx
host = 'http://localhost:9000'
async def getIndices(fromIsoInstant: str, toIsoInstant: str, catNames: [str]) -> pd.DataFrame:
    def listToStr(l):
        result = l[0]
        for i in range(1, len(l)):
            result += f", {l[i]}"
        return result    
    
    columns = ['indexId', 'isoInstant', 'indexValue', 'isRebalanceDay', 'divisor', 'lastRebalanceISOInstant', 'indexConstituents']
    indexIds = list(categs2IndexIds(catNames).values())
    sqlIn = f"\'{indexIds[0]}\'"
    for i in range(1, len(indexIds)):
        sqlIn += f", \'{indexIds[i]}\'"
    sqlIn = "(" + sqlIn + ")"
    sqlStmt = "SELECT %s FROM onn_daily2 WHERE isoInstant >= '%s' AND isoInstant < '%s' AND indexId IN %s"%(listToStr(columns), fromIsoInstant, toIsoInstant, sqlIn)
    # print(sqlStmt)
    query_params = {'query': sqlStmt, 'fmt': 'json'}
    async with httpx.AsyncClient() as client:
        r = await client.get(host + '/exec', params=query_params)
    jsonR = r.json()
    #print(jsonR)
    result = pd.DataFrame(columns=columns, data=jsonR['dataset'])
    return result

In [6]:
import json
import pandas as pd
class IndexData:
    indexId: str
    _indexValue: float
    isoInstant: str
    isRebalanceDay: bool
    _divisor: float
    lastRebalanceISOInstant: str
    indexConstituents: pd.DataFrame
    
    class IndexDataEncoder(json.JSONEncoder):
        def default(self, obj):
            if isinstance(obj, IndexData):
                return {"indexId": obj.indexId, "isoInstant": obj.isoInstant, "indexValue": obj.indexValue, "isRebalanceDay": obj.isRebalanceDay, "divisor": obj.divisor, "lastRebalanceISOInstant": obj.lastRebalanceISOInstant, "indexConstituents": json.loads(obj.indexConstituents.to_json(orient='split', index=False))}
            return json.JSONEncoder.default(self, obj)
    
    def set_indexValue(self, value):
        self._indexValue = round(value, 2)
    
    def get_indexValue(self):
        return self._indexValue
    
    def del_indexValue(self):
        del self._indexValue
        
    indexValue = property(get_indexValue, set_indexValue, del_indexValue)
    
    def set_divisor(self, value):
        self._divisor = round(value, 4)
    
    def get_divisor(self):
        return self._divisor
    
    def del_divisor(self):
        del self._divisor
       
    divisor = property(get_divisor, set_divisor, del_divisor)
    
    def toJSON(self):
        return json.dumps(self, cls=IndexData.IndexDataEncoder)
    
    def __repr__(self):
        return self.toJSON()
    
    def __jsonDecode(dct):
        attrs = ["indexId", "isoInstant", "indexValue", "isRebalanceDay", "divisor", "lastRebalanceISOInstant", "indexConstituents"]
        for attr in attrs:
            if attr not in dct:
                return dct
        result = IndexData()
        result.indexId = dct['indexId']
        result.isoInstant = dct['isoInstant']
        result.indexValue = dct['indexValue']
        result.isRebalanceDay = dct['isRebalanceDay']
        result.divisor = dct['divisor']
        result.lastRebalanceISOInstant = dct['lastRebalanceISOInstant']
        result.indexConstituents = pd.read_json(json.dumps(dct['indexConstituents']), orient='split')
        return result
    
    def fromJSON(jsStr):
        return json.loads(jsStr, object_hook=IndexData.__jsonDecode)

In [7]:
async def getDailyIndices(isoInstant: str, catNames: [str]) -> Dict[str, IndexData]:
    def listToStr(l):
        result = l[0]
        for i in range(1, len(l)):
            result += f", {l[i]}"
        return result    
    
    columns = ['indexId', 'isoInstant', 'indexValue', 'isRebalanceDay', 'divisor', 'lastRebalanceISOInstant', 'indexConstituents']
    indexIds = list(categs2IndexIds(catNames).values())
    sqlIn = f"\'{indexIds[0]}\'"
    for i in range(1, len(indexIds)):
        sqlIn += f", \'{indexIds[i]}\'"
    sqlIn = "(" + sqlIn + ")"
    sqlStmt = "SELECT %s FROM onn_daily2 WHERE isoInstant = '%s' AND indexId IN %s"%(listToStr(columns), isoInstant, sqlIn)
    # print(sqlStmt)
    query_params = {'query': sqlStmt, 'fmt': 'json'}
    async with httpx.AsyncClient() as client:
        r = await client.get(host + '/exec', params=query_params)
    jsonR = r.json()
    # print(jsonR)
    resultArray = pd.DataFrame(columns=columns, data=jsonR['dataset'])
    resultArray = resultArray.to_dict('records')
    result = {}
    for rec in resultArray:
        rec['indexConstituents'] = json.loads(rec['indexConstituents'])
        result[rec['indexId']] = IndexData.fromJSON(json.dumps(rec))
    return result

In [8]:
import pandas as pd
import httpx
host = 'http://localhost:9000'
async def getSymb(fromIsoInstant: str, toIsoInstant: str, symb: str) -> pd.DataFrame:
    def listToStr(l):
        result = l[0]
        for i in range(1, len(l)):
            result += f", {l[i]}"
        return result
    
    columns = ['symb', 'open_time', 'close', 'volume', 'close_time', 'quote_asset_vol']        
    sqlStmt = "SELECT %s FROM 'binance_daily' WHERE symb = '%s' AND open_time >= '%s' AND open_time < '%s'"%(listToStr(columns), symb, fromIsoInstant, toIsoInstant)

    query_params = {'query': sqlStmt, 'fmt': 'json'}
    async with httpx.AsyncClient() as client:
        r = await client.get(host + '/exec', params=query_params)
    jsonR = r.json()
    #print(jsonR)
    result = pd.DataFrame(columns=columns, data=jsonR['dataset'])
    result['strDate'] = result.apply(lambda row: row['open_time'][0:10], axis = 1)
    result.set_index('strDate', drop=True, inplace=True)
    result.rename(columns={'close':symb}, inplace=True)
    result = result[[symb]].sort_values('strDate')
    
    return result    

In [9]:
btcDF = await getSymb('2020-12-01T00:00:00.000000Z', '2022-08-30T00:00:00.000000Z', 'BTCUSDT')
btcDF.rename(columns={'BTCUSDT':'BTC'}, inplace=True)
dfBTCIndex = btcDF[['BTC']].sort_values('strDate')
dfBTCIndex['BTC'] = 1000*(dfBTCIndex['BTC']/(dfBTCIndex['BTC'][0]))
del btcDF

In [10]:
dfBTCIndex.head()

,BTC
strDate,
2020-12-01,1000.000000
2020-12-02,1023.401595
2020-12-03,1035.008868
2020-12-04,993.901399
2020-12-05,1020.394395


In [11]:
import riskfolio as rp
from dateutil.parser import parse
import datetime
from typing import Dict

async def computeOptimalWeights(isoInstant: str, categNames: [str]) -> Dict[str, float]:
    method_mu='hist' # Method to estimate expected returns based on historical data.
    method_cov='hist' # Method to estimate covariance matrix based on historical data.
    model='Classic' # Could be Classic (historical), BL (Black Litterman) or FM (Factor Model)
    rm = 'MV' # Risk measure used, this time will be variance
    #rm = 'MDD' # Maximum Drawdown of uncompounded cumulative returns (Calmar Ratio)
    #obj = 'Sharpe' # Objective function, could be MinRisk, MaxRet, Utility or Sharpe
    obj = 'Sharpe'
    hist = True # Use historical scenarios for risk measures that depend on scenarios
    rf = 0 # Risk free rate
    l = 0 # Risk aversion factor, only useful when obj is 'Utility'

    currentDay = parse(str(isoInstant))
    windowStart = currentDay - datetime.timedelta(days = 30)
    
    result = await getIndices(windowStart, str(currentDay), categNames)
    result["Date"] = result.isoInstant.astype('datetime64[ns]')
    result = result.pivot(index="Date", columns=["indexId"], values=["indexValue"])
    result.columns = result.columns.droplevel()
    result = result.reset_index()
    result['strDate'] = result['Date'].apply(lambda val : (val - datetime.timedelta(seconds=1)).strftime('%Y-%m-%d'))
    result.set_index('strDate', drop=True, inplace=True)
    result.drop('Date', axis=1, inplace=True)
    result = result.dropna(axis=1, how="any")
    if 'bitcoin' in categNames:
        result = pd.concat([result, dfBTCIndex], join='inner', axis=1).sort_values('strDate')
    if 'USDCUSDT' in categNames:
        usdcDF = await getSymb(str(windowStart), str(currentDay), 'USDCUSDT')
        result = pd.concat([result, usdcDF], join='inner', axis=1).sort_values('strDate')
    result.rename(index={'strDate':'Date'}, inplace=True)
        
    Y = result.pct_change().dropna()
    # Building the portfolio object
    port = rp.Portfolio(returns=Y)
    port.assets_stats(method_mu=method_mu, method_cov=method_cov, d=0.94)
    w = port.optimization(model=model, rm=rm, obj=obj, rf=rf, l=l, hist=hist)
    # display(w)
    if w is None:
        return None
    else:
        result = {}
        categIndices = categs2IndexIds(categNames)

        for cat, indexName in categIndices.items():
            if indexName in w.index:
                weight = w.loc[indexName][0]
                if weight < 0.001:
                    weight = 0
            else:
                weight = 0
            result[cat] = weight
        if 'bitcoin' in categNames:
            result['bitcoin'] = w.loc['BTC'][0]
        if 'USDCUSDT' in categNames:
            result['USDCUSDT'] = w.loc['USDCUSDT'][0]
        return result

In [12]:
from typing import Dict
from dateutil.parser import parse
import datetime

#falta levar em consideração o perdido mensalmente com rebalance dos índices, mas isso sera feito so quando for comprar as assets subjacentes
async def simulateDailyRebal(startValue: float, isoInstantStart: str, isoInstantEnd: str, 
                             categNames: [str], discountSell: float, 
                             discountBuy: float) -> Dict[str, object]:
    wallet = {}
    wallet['portfolio'] = {}
    wallet['totalValue'] = 0.0
    wallet['pocketMoney'] = 0.0

    async def categIndexVals(categNames: [str], isoInstant) -> Dict[str, float]:
        categIndexIds = categs2IndexIds(categNames)
        indices = await getDailyIndices(str(isoInstant), categNames)
        result = {}
        for cat in categNames:
            if cat == 'bitcoin':
                indexVal = float(dfBTCIndex.loc[isoInstant[:10]])
            elif cat == 'USDCUSDT':
                indexVal = await getSymb(str(isoInstant), str(parse(isoInstant) + datetime.timedelta(days = 1)), 'USDCUSDT')
                indexVal = indexVal['USDCUSDT'][0]
            else:
                indexId = categIndexIds[cat]
                indices = await getDailyIndices(isoInstant, categNames)
                if indexId in indices:
                    indexData = indices[indexId]
                    indexVal = indexData.indexValue
                else:
                    indexVal = 0
            result[cat] = indexVal
        return result
    
    lostToRebal = startValue*discountBuy
    startIndices = await categIndexVals(categNames, isoInstantStart)
    categWeights = await computeOptimalWeights(isoInstantStart, categNames)
    for cat in categWeights:
        buyCat = categWeights[cat]*startValue
        buyCat *= 1.0 - discountSell
        print(startIndices)
        if(buyCat > 0):
            wallet['portfolio'][cat] = buyCat/startIndices[cat]
            wallet['totalValue'] += buyCat
        else:
            wallet['portfolio'][cat] = 0
    result = {}
    result['days'] = {}
    result['days'][isoInstantStart] = wallet
    print(f"{isoInstantStart},{wallet}")
    currentDay = parse(isoInstantStart) + datetime.timedelta(days = 1)
    while (currentDay < parse(isoInstantEnd)):
        print(f"Today is {currentDay}")
        currIndices = await categIndexVals(categNames, str(currentDay))
        currWalletVals = {}
        totalValue = 0.0
        for cat in wallet['portfolio']:
            currWalletVals[cat] = wallet['portfolio'][cat]*currIndices[cat]
            totalValue += currWalletVals[cat]
        pocketMoney = 0.0
        if 'pocketMoney' in wallet:
            pocketMoney += wallet['pocketMoney']
            
        newWeights = await computeOptimalWeights(currentDay, categNames)
        print(f"START wallet (assetValue, pocketMoney): ({totalValue}, {pocketMoney})")
        
        if newWeights is not None:
            print(newWeights)
            categWeights = newWeights
            #selling excess
            excessCats = set()
            newPocketMoney = pocketMoney
            newTotalValue = totalValue
            for cat in wallet['portfolio']:
                excessWeight = (currWalletVals[cat]/(totalValue+pocketMoney)) - categWeights[cat]
                if excessWeight > 0:
                    excessCats.add(cat)
                    sellAmount = excessWeight*(totalValue+pocketMoney) #will sell this amount
                    newPocketMoney += sellAmount*(1.0 - discountSell) #but will receive less trade tax
                    currWalletVals[cat] -= sellAmount
                    lostToRebal += sellAmount*(discountSell) 
                    newTotalValue -= sellAmount
            notExcessCats = set(wallet['portfolio'].keys()) - excessCats
            for cat in notExcessCats:
                missingWeight = categWeights[cat] - (currWalletVals[cat]/(totalValue+pocketMoney))
                if missingWeight > 0:
                    buyAmount = missingWeight*(totalValue+pocketMoney)
                    if newPocketMoney < buyAmount:
                        print(f"Warning: {newPocketMoney} of new pocket money cannot buy {buyAmount}, buying MAX")
                        buyAmount = newPocketMoney
                    currWalletVals[cat] += buyAmount*(1.0 - discountBuy)
                    newPocketMoney -= buyAmount
                    lostToRebal += buyAmount*discountBuy
                    newTotalValue += buyAmount*(1.0 - discountBuy)
            newWallet = {}
            newWallet['totalValue'] = newTotalValue
            newWallet['pocketMoney'] = newPocketMoney
            newWallet['portfolio'] = {}
            for cat in categWeights:
                if currIndices[cat] != 0:
                    newWallet['portfolio'][cat] = currWalletVals[cat]/currIndices[cat]
                else:
                    newWallet['portfolio'][cat] = 0
        else:
            print("WARNING: inexistent optimal portfolio, keeping stablecoins")
            newWallet = {}
            newWallet['totalValue'] = 0
            newWallet['pocketMoney'] = pocketMoney + (totalValue*(1.0 - discountSell))
            newWallet['portfolio'] = {}
            for cat in categWeights:
                newWallet['portfolio'][cat] = 0
            
        
        wallet = newWallet
        result['days'][str(currentDay)] = wallet
        print(f"{currentDay},{wallet}")
        currentDay += datetime.timedelta(days = 1)
    result['totalLostToRebal'] = lostToRebal
    return result

In [13]:
result = await simulateDailyRebal(1000, '2021-01-01T00:00:00.000000Z', "2022-08-30T00:00:00.000000Z", categories, 0.001, 0.001)

{'bitcoin': 1563.1096469163806, 'smart-contract-platform': 1139.46, 'centralized-exchange-token-cex': 1190.09, 'decentralized-finance-defi': 971.8, 'non-fungible-tokens-nft': 2371.51, 'meme-token': 1296.95, 'gaming': 867.75, 'Other': 675.68, 'sports': 0}
{'bitcoin': 1563.1096469163806, 'smart-contract-platform': 1139.46, 'centralized-exchange-token-cex': 1190.09, 'decentralized-finance-defi': 971.8, 'non-fungible-tokens-nft': 2371.51, 'meme-token': 1296.95, 'gaming': 867.75, 'Other': 675.68, 'sports': 0}
{'bitcoin': 1563.1096469163806, 'smart-contract-platform': 1139.46, 'centralized-exchange-token-cex': 1190.09, 'decentralized-finance-defi': 971.8, 'non-fungible-tokens-nft': 2371.51, 'meme-token': 1296.95, 'gaming': 867.75, 'Other': 675.68, 'sports': 0}
{'bitcoin': 1563.1096469163806, 'smart-contract-platform': 1139.46, 'centralized-exchange-token-cex': 1190.09, 'decentralized-finance-defi': 971.8, 'non-fungible-tokens-nft': 2371.51, 'meme-token': 1296.95, 'gaming': 867.75, 'Other': 6

/home/srmq/devel/onn-indices/.venv/lib/python3.8/site-packages/cvxpy/problems/problem.py:1337: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


START wallet (assetValue, pocketMoney): (19147.923411558324, 1.7620930411781983e-07)
{'smart-contract-platform': 0, 'centralized-exchange-token-cex': 0, 'decentralized-finance-defi': 0, 'non-fungible-tokens-nft': 0, 'meme-token': 0.999999999974848, 'gaming': 0, 'Other': 0, 'sports': 0, 'bitcoin': 4.5022611643620677e-13}
2021-06-04 00:00:00+00:00,{'totalValue': 19147.92341126155, 'pocketMoney': 4.7268947343266716e-07, 'portfolio': {'smart-contract-platform': 0.0, 'centralized-exchange-token-cex': 0.0, 'decentralized-finance-defi': 0.0, 'non-fungible-tokens-nft': 0.0, 'meme-token': 0.17169599260464102, 'gaming': 0.0, 'Other': 0.0, 'sports': 0, 'bitcoin': 4.392482921269481e-12}}
Today is 2021-06-05 00:00:00+00:00
The problem doesn't have a solution with actual input parameters
START wallet (assetValue, pocketMoney): (17978.960433931283, 4.7268947343266716e-07)
2021-06-05 00:00:00+00:00,{'totalValue': 0, 'pocketMoney': 17960.98147397004, 'portfolio': {'smart-contract-platform': 0, 'central

/home/srmq/devel/onn-indices/.venv/lib/python3.8/site-packages/cvxpy/problems/problem.py:1337: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


START wallet (assetValue, pocketMoney): (0.0, 17960.98147397004)
{'smart-contract-platform': 0, 'centralized-exchange-token-cex': 0, 'decentralized-finance-defi': 0, 'non-fungible-tokens-nft': 0.9999998803860468, 'meme-token': 0, 'gaming': 0, 'Other': 0, 'sports': 0, 'bitcoin': 2.677595832638822e-10}
2021-06-12 00:00:00+00:00,{'totalValue': 17943.018351064875, 'pocketMoney': 0.0021435747694340535, 'portfolio': {'smart-contract-platform': 0.0, 'centralized-exchange-token-cex': 0.0, 'decentralized-finance-defi': 0.0, 'non-fungible-tokens-nft': 1.5438546462087672, 'meme-token': 0.0, 'gaming': 0.0, 'Other': 0.0, 'sports': 0, 'bitcoin': 2.5362738211882896e-09}}
Today is 2021-06-13 00:00:00+00:00
The problem doesn't have a solution with actual input parameters
START wallet (assetValue, pocketMoney): (17368.164074018652, 0.0021435747694340535)
2021-06-13 00:00:00+00:00,{'totalValue': 0, 'pocketMoney': 17350.798053519404, 'portfolio': {'smart-contract-platform': 0, 'centralized-exchange-token-

...

START wallet (assetValue, pocketMoney): (13150.698631207582, 3.531789837971817e-06)
{'smart-contract-platform': 0, 'centralized-exchange-token-cex': 0, 'decentralized-finance-defi': 0, 'non-fungible-tokens-nft': 0, 'meme-token': 0.9999999986108172, 'gaming': 0, 'Other': 0, 'sports': 0, 'bitcoin': 1.4475107900792217e-10}
2022-04-26 00:00:00+00:00,{'totalValue': 13150.698618372599, 'pocketMoney': 1.6350682693487527e-05, 'portfolio': {'smart-contract-platform': 0.0, 'centralized-exchange-token-cex': 0.0, 'decentralized-finance-defi': 0.0, 'non-fungible-tokens-nft': 0.0, 'meme-token': 0.2159488314910343, 'gaming': 0.0, 'Other': 0.0, 'sports': 0, 'bitcoin': 9.36434281886188e-10}}
Today is 2022-04-27 00:00:00+00:00
START wallet (assetValue, pocketMoney): (11826.664504837552, 1.6350682693487527e-05)
{'smart-contract-platform': 0, 'centralized-exchange-token-cex': 0, 'decentralized-finance-defi': 0, 'non-fungible-tokens-nft': 0, 'meme-token': 0.9999999995890033, 'gaming': 0, 'Other': 0, 'sport

/home/srmq/devel/onn-indices/.venv/lib/python3.8/site-packages/cvxpy/problems/problem.py:1337: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


START wallet (assetValue, pocketMoney): (0.0, 10466.163734753834)
{'smart-contract-platform': 0, 'centralized-exchange-token-cex': 0, 'decentralized-finance-defi': 0, 'non-fungible-tokens-nft': 0, 'meme-token': 0.9999999999291196, 'gaming': 0, 'Other': 0, 'sports': 0, 'bitcoin': 4.445650577134515e-12}
2022-07-09 00:00:00+00:00,{'totalValue': 10455.697570324459, 'pocketMoney': 6.953159754630178e-07, 'portfolio': {'smart-contract-platform': 0.0, 'centralized-exchange-token-cex': 0.0, 'decentralized-finance-defi': 0.0, 'non-fungible-tokens-nft': 0.0, 'meme-token': 0.3848774978660317, 'gaming': 0.0, 'Other': 0.0, 'sports': 0, 'bitcoin': 4.0396759457939124e-11}}
Today is 2022-07-10 00:00:00+00:00


/home/srmq/devel/onn-indices/.venv/lib/python3.8/site-packages/cvxpy/problems/problem.py:1337: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


START wallet (assetValue, pocketMoney): (10658.512616598373, 6.953159754630178e-07)
{'smart-contract-platform': 0, 'centralized-exchange-token-cex': 0, 'decentralized-finance-defi': 0, 'non-fungible-tokens-nft': 0, 'meme-token': 0.9981355338280405, 'gaming': 0, 'Other': 0, 'sports': 0, 'bitcoin': 0.0013099425701407709}
2022-07-10 00:00:00+00:00,{'totalValue': 10652.588258447751, 'pocketMoney': 5.890524371092701, 'portfolio': {'smart-contract-platform': 0.0, 'centralized-exchange-token-cex': 0.0, 'decentralized-finance-defi': 0.0, 'non-fungible-tokens-nft': 0.0, 'meme-token': 0.38415990681759177, 'gaming': 0.0, 'Other': 0.0, 'sports': 0, 'bitcoin': 0.012545739501130675}}
Today is 2022-07-11 00:00:00+00:00
START wallet (assetValue, pocketMoney): (10303.450546554815, 5.890524371092701)
{'smart-contract-platform': 0, 'centralized-exchange-token-cex': 0, 'decentralized-finance-defi': 0, 'non-fungible-tokens-nft': 0, 'meme-token': 0.9999999999663938, 'gaming': 0, 'Other': 0, 'sports': 0, 'bi

In [14]:
import csv
with open('Historical-ONN-PortfolioOptDaily-B-30dW-30dR_2020-12-01_2022-08-30.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['Date', 'Close'])
    for key in result['days']:
        vals = []
        vals.append(key)
        vals.append(result['days'][key]['totalValue']+result['days'][key]['pocketMoney'])
        writer.writerow(vals)
